In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from vgg16 import *
from tensorflow.python import debug as tf_debug 

In [2]:
def _image_parser (path, label):
    
    # Convert One Hot Label
    # one_hot_label = tf.one_hot(label, NUM_CLASSES)
    
    # Read Image
    image_file = tf.read_file(path)
    image_decoded = tf.image.decode_jpeg(image_file, channels=3)
    image_resized = tf.image.resize_images(image_decoded, INPUT_IMAGE_SIZE)

    #return image_resized, one_hot_label
    return image_resized, label

In [3]:
# Misc
LOG_DIR = './log/'
MODEL_ID = 1
IMGNET_PRE_MODEL = './imagenet_pretrained_20/model.ckpt'

# Training Parameters
MEAN_IMAGE = np.array([104., 117., 124.])
SHUFFLE_BUFFER= 250
LEARNING_RATE = 0.005
MOMENTUM = 0.9
NUM_EPOCHS = 1
NUM_STEPS = 5000
BATCH_SIZE = 10
DISPLAY_STEP = 1
INPUT_IMAGE_SIZE = [256, 256]
NUM_CLASSES = 20
KEEP_RATE = 0.75

In [18]:
# Data preparation
train = pd.read_csv('../train20.txt', delimiter=' ', header=None).sample(frac=1)  

In [19]:
train = train.head(10)

In [27]:
# Dataset
trn_ds = tf.data.Dataset.from_tensor_slices(
    (np.array(train[0]), np.array(train[1]))).map(_image_parser).batch(BATCH_SIZE)

iterator = trn_ds.make_one_shot_iterator()
x, y = iterator.get_next()

# Crop and resize
crop_resize = tf.image.crop_and_resize(
    image=x,
    boxes=[[0, 0, 0.9, 1]] * BATCH_SIZE,
    box_ind=range(BATCH_SIZE),
    crop_size=[int(x.get_shape()[1]), 
        int(x.get_shape()[2])],
    name='crop_resize')

# Convert to BGR <-- Because the weights were trained from opencv images
bgr = tf.reverse(
    tensor=crop_resize, 
    axis=[-1],
    name='bgr')

# Subtract from mean
bgr_sub = tf.subtract(
    x=bgr,
    y=MEAN_IMAGE,
    name='mean_subtraction')

# Crop
final_input = tf.image.resize_image_with_crop_or_pad(
        image=bgr_sub,
        target_height=224,
        target_width=224)

random_crop = tf.random_crop(
    value=x,
    size=[BATCH_SIZE, 80, 80, 3])

# Image Summary
summary_op = tf.summary.image('input', random_crop, BATCH_SIZE)    

In [28]:
with tf.Session() as sess:
    # Run
    summary, lab = sess.run([summary_op, y])
    print(lab)
    # Write summary
    writer = tf.summary.FileWriter('./log')
    writer.add_summary(summary)
    writer.close()

[ 0  6  5 15 10  9  3 11  5 19]


In [26]:
with tf.Session() as sess:
    # Run
    summary, lab = sess.run([summary_op, y])
    print(lab)
    # Write summary
    writer = tf.summary.FileWriter('./log')
    writer.add_summary(summary)
    writer.close()

[ 0  6  5 15 10  9  3 11  5 19]


In [6]:
np.array(train[0])

array([ '/home/public/dataset/ecology/Mozambique/Hartebeest/2016:11:23_12:06:36.jpg',
       '/home/public/dataset/ecology/Mozambique/Impala/2016_10_04_14_53_17.jpg',
       '/home/public/dataset/ecology/Mozambique/Porcupine/2016_08_07_18_11_37.jpg',
       '/home/public/dataset/ecology/Mozambique/Hare/2016_07_13_22_14_15.jpg',
       '/home/public/dataset/ecology/Mozambique/Vervet/2016_07_01_12_22_49.jpg',
       '/home/public/dataset/ecology/Mozambique/Genet/2016_07_19_19_45_03.jpg',
       '/home/public/dataset/ecology/Mozambique/Nyala/2016_09_29_06_25_32.jpg',
       '/home/public/dataset/ecology/Mozambique/Elephant/2016:12:30_04:55:08.jpg',
       '/home/public/dataset/ecology/Mozambique/Kudu/2016_09_12_16_23_38.jpg_original.jpg',
       '/home/public/dataset/ecology/Mozambique/Waterbuck/2016_07_08_14_12_32.jpg'], dtype=object)

In [16]:
# Read Image
image_file = tf.read_file('/home/public/dataset/ecology/Mozambique/Hartebeest/2016:11:23_12:06:36.jpg')
image_decoded = tf.image.decode_jpeg(image_file, channels=3)
image_resized = tf.image.resize_images(image_decoded, INPUT_IMAGE_SIZE)

random_crop = tf.random_crop(
    value=image_resized,
    size=[180, 180, 3])

image = tf.expand_dims(random_crop, 0)

# Image Summary
summary_op = tf.summary.image('input', image) 

In [17]:
with tf.Session() as sess:
    # Run
    summary= sess.run(summary_op)
    # Write summary
    writer = tf.summary.FileWriter('./log')
    writer.add_summary(summary)
    writer.close()

In [33]:
[[0, 0, 0.9, 1]] * 10

[[0, 0, 0.9, 1],
 [0, 0, 0.9, 1],
 [0, 0, 0.9, 1],
 [0, 0, 0.9, 1],
 [0, 0, 0.9, 1],
 [0, 0, 0.9, 1],
 [0, 0, 0.9, 1],
 [0, 0, 0.9, 1],
 [0, 0, 0.9, 1],
 [0, 0, 0.9, 1]]